# Building a Fraud Prediction Model with EvalML

In this demo, we will build an optimized fraud prediction model using EvalML. To optimize the model we will set up an objective function based on some assumptions about our business. 

In [1]:
%load_ext autoreload
%autoreload 2
import evalml
from evalml.objectives import FraudCost
from evalml.preprocessing import split_data

## Load Transactions

In [2]:
X, y = evalml.demos.load_fraud()

             Number of Features
Boolean                       1
Categorical                   6
Numeric                       5

Number of training examples: 99992

Labels
False    84.82%
True     15.18%
Name: fraud, dtype: object


## Define Objective

In [3]:
fraud_objective = FraudCost(
    retry_percentage=.5,
    interchange_fee=.02,
    fraud_payout_percentage=.75,
    amount_col='amount'
)

## Search for best modeling pipeline

In order to validate the results of the pipeline creation and optimization process, we will save some of our data as a holdout set

In [4]:
numerics = ['int16', 'int32', 'int64', 'float16', 'float32', 'float64', 'bool']
X = X.select_dtypes(include=numerics)
X_train, X_holdout, y_train, y_holdout = split_data(X, y, test_size=.8, random_state=0)

Because the fraud labels are binary, we will use `AutoClassifier`. When we call `.fit()`, the search for the best pipeline will begin. 

In [5]:
clf = evalml.AutoClassifier(objective=fraud_objective,
                            max_time=120,
                            max_pipelines=20)

clf.fit(X_train, y_train)

*****************************
* Beginning pipeline search *
*****************************

Optimizing for Fraud Cost. Lower score is better.

Searching up to 20 pipelines. 
Will stop searching for new pipelines after 120 seconds.

Possible model types: random_forest, linear_model, xgboost

✔ XGBoost Classifier w/ One Hot Encod...     0%|          | Elapsed:00:02
▹ XGBoost Classifier w/ One Hot Encod...     5%|▌         | Elapsed:00:02

/Users/angela.lin/Desktop/env/lib/python3.7/site-packages/sklearn/metrics/classification.py:872: RuntimeWarning: invalid value encountered in double_scalars
  mcc = cov_ytyp / np.sqrt(cov_ytyt * cov_ypyp)
/Users/angela.lin/Desktop/env/lib/python3.7/site-packages/sklearn/metrics/classification.py:872: RuntimeWarning: invalid value encountered in double_scalars
  mcc = cov_ytyp / np.sqrt(cov_ytyt * cov_ypyp)


✔ XGBoost Classifier w/ One Hot Encod...     5%|▌         | Elapsed:00:05
▹ Random Forest Classifier w/ One Hot...    10%|█         | Elapsed:00:05

/Users/angela.lin/Desktop/env/lib/python3.7/site-packages/sklearn/metrics/classification.py:872: RuntimeWarning: invalid value encountered in double_scalars
  mcc = cov_ytyp / np.sqrt(cov_ytyt * cov_ypyp)


✔ Random Forest Classifier w/ One Hot...    10%|█         | Elapsed:00:39
▹ XGBoost Classifier w/ One Hot Encod...    15%|█▌        | Elapsed:00:39

...

✔ Random Forest Classifier w/ One Hot...    60%|██████    | Elapsed:01:47
▹ Logistic Regression Classifier w/ O...    65%|██████▌   | Elapsed:01:47

/Users/angela.lin/Desktop/env/lib/python3.7/site-packages/sklearn/metrics/classification.py:872: RuntimeWarning: invalid value encountered in double_scalars
  mcc = cov_ytyp / np.sqrt(cov_ytyt * cov_ypyp)


✔ Logistic Regression Classifier w/ O...    65%|██████▌   | Elapsed:01:56
✔ XGBoost Classifier w/ One Hot Encod...    70%|███████   | Elapsed:01:57
▹ Random Forest Classifier w/ One Hot...    75%|███████▌  | Elapsed:01:57

/Users/angela.lin/Desktop/env/lib/python3.7/site-packages/sklearn/metrics/classification.py:872: RuntimeWarning: invalid value encountered in double_scalars
  mcc = cov_ytyp / np.sqrt(cov_ytyt * cov_ypyp)


✔ Random Forest Classifier w/ One Hot...    75%|███████▌  | Elapsed:02:19
✔ Random Forest Classifier w/ One Hot...    80%|████████  | Elapsed:02:19


Max time elapsed. Stopping search early.

✔ Optimization finished


### View rankings and select pipeline

Once the fitting process is done, we can see all of the pipelines that were searched, ranked by their score on the objective function we defined

In [6]:
clf.rankings

,id,pipeline_name,score,high_variance_cv,parameters
0,14,XGBoostPipeline,0.007614,False,"{'eta': 0.264555612104627, 'min_child_weight':..."
1,12,RFClassificationPipeline,0.007614,False,"{'n_estimators': 369, 'max_depth': 10, 'impute..."
2,2,RFClassificationPipeline,0.007615,False,"{'n_estimators': 569, 'max_depth': 22, 'impute..."
3,0,XGBoostPipeline,0.007615,False,"{'eta': 0.5928446182250184, 'min_child_weight'..."
4,3,XGBoostPipeline,0.007615,False,"{'eta': 0.5288949197529046, 'min_child_weight'..."
5,5,XGBoostPipeline,0.007615,False,"{'eta': 0.6481718720511973, 'min_child_weight'..."
6,10,XGBoostPipeline,0.007615,False,"{'eta': 0.7206326547259169, 'min_child_weight'..."
7,7,XGBoostPipeline,0.007615,False,"{'eta': 0.9786183422327642, 'min_child_weight'..."
8,1,XGBoostPipeline,0.007616,False,"{'eta': 0.38438170729269994, 'min_child_weight..."
9,15,RFClassificationPipeline,0.007616,False,"{'n_estimators': 609, 'max_depth': 7, 'impute_..."


to select the best pipeline we can run

In [7]:
pipeline = clf.best_pipeline

to select another pipeline we can use the id

In [8]:
pipeline = clf.get_pipeline(0)

### Describe pipeline

You can get more details about any pipeline. Including 

In [9]:
clf.describe_pipeline(0)

********************************************************************************************
* XGBoost Classifier w/ One Hot Encoder + Simple Imputer + RF Classifier Select From Model *
********************************************************************************************

Problem Types: Binary Classification, Multiclass Classification
Model Type: XGBoost Classifier
Objective to Optimize: Fraud Cost (lower is better)
Number of features: 3

Pipeline Steps
1. One Hot Encoder
2. Simple Imputer
	 * impute_strategy : most_frequent
3. RF Classifier Select From Model
	 * percent_features : 0.6273280598181127
	 * threshold : -inf
4. XGBoost Classifier
	 * eta : 0.5928446182250184
	 * max_depth : 4
	 * min_child_weight : 8.598391737229157

Training
Training for Binary Classification problems.
Total training time (including CV): 2.0 seconds

Cross Validation
----------------
             Fraud Cost    F1  Precision  Recall   AUC  Log Loss    MCC # Training # Testing
0                 0.008